<a href="https://colab.research.google.com/github/dcpatton/Image-Classification/blob/master/cifar10_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
import random

seed = 52
tf.random.set_seed(seed)
random.seed(seed)

tf.__version__

'2.3.0'

In [14]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
# Normalize the data
train_images, test_images = train_images / 255.0, test_images / 255.0
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [15]:
print(train_images.shape)
print(test_images.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [16]:
from tensorflow.keras.layers import Conv2D, Flatten, MaxPool2D, Dense
from tensorflow.keras import Input, Model

inp = Input(shape=(32, 32, 3), name='inp')
x = Conv2D(32, 3, activation='relu')(inp)
x = MaxPool2D()(x)
x = Conv2D(64, 3, activation='relu')(x)
x = MaxPool2D()(x)
x = Conv2D(128, 3, activation='relu')(x)
x = MaxPool2D()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)
out = Dense(10, activation='softmax', name='out')(x)

model = Model(inputs=[inp], outputs=[out])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), 
              metrics=['acc'])

model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 2, 2, 128)        

In [17]:
filepath = 'model.h5'
mc = tf.keras.callbacks.ModelCheckpoint(filepath, save_best_only=True, 
                                        save_weights_only=True)
es = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1)

history = model.fit(train_images, train_labels, epochs=200, 
                    validation_data=(test_images, test_labels), 
                    callbacks=[mc, es], verbose=1)

Epoch 1/200
1563/1563 [==============================] - 7s 5ms/step - loss: 1.4951 - acc: 0.4511 - val_loss: 1.2087 - val_acc: 0.5695
Epoch 2/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1001 - acc: 0.6108 - val_loss: 1.0607 - val_acc: 0.6267
Epoch 3/200
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9210 - acc: 0.6758 - val_loss: 0.9197 - val_acc: 0.6781
Epoch 4/200
1563/1563 [==============================] - 6s 4ms/step - loss: 0.8051 - acc: 0.7181 - val_loss: 0.9093 - val_acc: 0.6849
Epoch 5/200
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7148 - acc: 0.7509 - val_loss: 0.8366 - val_acc: 0.7126
Epoch 6/200
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6379 - acc: 0.7759 - val_loss: 0.8537 - val_acc: 0.7127
Epoch 7/200
1563/1563 [==============================] - 6s 4ms/step - loss: 0.5753 - acc: 0.7981 - val_loss: 0.8368 - val_acc: 0.7158
Epoch 8/200
1563/1563 [==============================] 

In [18]:
model.load_weights(filepath)
metrics = model.evaluate(test_images, test_labels, verbose=0)
print('Loss = ' + str(metrics[0]))
print('Accuracy = ' + str(metrics[1]))

Loss = 0.8365728259086609
Accuracy = 0.7125999927520752
